In [14]:
%pip install lobsang openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


# Basics

In this notebook, we'll cover the basics of lobsang. As a LLM we use OpenAI's "gpt-3.5-turbo" model, so you'll need an [OpenAI API key](https://platform.openai.com/account/api-keys) to run this notebook (set it in the cell below).
This is not a free service, but all examples in this notebook should cost less than $0.01 to run. Nevertheless, you should set a [usage limit](https://platform.openai.com/account/billing/limits) to avoid any surprises.

In [15]:
import os

from dotenv import load_dotenv
from openai import ChatCompletion

from lobsang import Chat, LLM
from lobsang.answers import TextAnswer
from lobsang.messages import SystemMessage, UserMessage, AssistantMessage

load_dotenv()

# Load OpenAI API key from .env file (please update .env file with your own API key)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Please set OPENAI_API_KEY in .env file"

f"All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY={OPENAI_API_KEY[:15]}..."

"All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY=sk-PbRY4M6AH6Xh..."

## Message Class

Before we get started, let's take a look at the `Message` class. This is the abstract base class for all messages in lobsang. It has three subclasses: `SystemMessage`, `UserMessage`, and `AssistantMessage`.

1. `SystemMessage` is a special case and is used to set the model's behavior (There should only be one `SystemMessage` per `Chat` instance at the beginning of the conversation.)
1. `UserMessage` is used for messages that are created by the user/developer.
1. `AssistantMessage` is used for messages that are sent generated by the llm.

In [16]:
system_message = SystemMessage("You are a helpful assistant.")
user_message = UserMessage("Hello, my name is Bark Twain.")
assistant_message = AssistantMessage("Hello Bark, I'm Lobsang.")

messages = [system_message, user_message, assistant_message]

print("__repr__")
print(*map(repr, messages), sep="\n")

__repr__
SystemMessage(text='You are a helpful assistant.', info={})
UserMessage(text='Hello, my name is Bark Twain.', info={})
AssistantMessage(text="Hello Bark, I'm Lobsang.", info={}, data=None)


☝️ As you can see, each message has a `text` and `info` attribute. 
2. `text` is the text of the message
3. `info` is a dictionary that can be used to store additional information about the message

👇 In, addition you access the role of the message via the `role` attribute:

In [17]:
user_message = UserMessage("Hello, my name is Bark Twain.")
print(f"role: {user_message.role}")

role: user


Also, the assistant message has an additional property `data` that contains the parsed response from the llm (we'll get to that in a bit):

In [18]:
assistant_message = AssistantMessage("Hello Bark, I'm Lobsang.")

print(assistant_message.data)  # 👈 This will be empty for now as we haven't run the message through an llm yet

None


## LLM Wrapper

For experiments and testing, we also provide a FakeLLM which can be imported via `from lobsang.llms import FakeLLM`. However, in this notebook we'll use OpenAI's API. So before we can start chatting, we need to wrap the openai api to make it compatible with lobsang. This can be done with lobsang's `LLM` class as follows:

In [19]:
class OpenAI(LLM):
    """Wrapper for OpenAI API"""

    def __init__(self):
        self.api_key = OPENAI_API_KEY
        self.model = "gpt-3.5-turbo"

    def chat(self, messages) -> (str, dict):
        """
        Chat with OpenAI API
        
        Takes a list of messages and converts them to the format required by the OpenAI API.
        Then, calls the API and retrieves the response text to return it to the caller.
        """
        # Prepare messages
        messages = [{'role': m.role, 'content': m.text} for m in messages]

        # Call OpenAI API
        res = ChatCompletion.create(
            model=self.model,
            api_key=self.api_key,
            messages=messages,
        )

        # Return text response
        return res.choices[0].message.content, {}

**Note**: In this example, we return an empty dictionary as the second return value. 
However, all keys added to the dictionary will end up in the `info` attribute of the corresponding `AssistantMessage` that is created by the chat instance from the response text. 
This can be used to store additional information about the message. 
This implementation is also provided under `lobsang.llms.openai` and can be imported via `from lobsang import OpenAI`.

## Chat Instance

Now, let's bring all the pieces together and create a `Chat` instance. 

In [20]:
chat = Chat(llm=OpenAI())

# Let's check the system message
print(chat.sys_msg)

SYSTEM: You are a helpful assistant.


☝️ As you can see, the chat has a default system message (which you could override during initialization).
 
👇 Before we start chatting, we want to change the LLM's behavior by updating the system message. 
Let's make it a fancy wizard 🧙‍♂️ that speaks in a medieval style:

In [21]:
# First, we need to write a system message that sets the model's behavior
system_message = SystemMessage("You are a fancy wizard and talk in a medieval style.")

# Then, we update the chat instance with the new system message
chat.sys_msg = system_message

# And we are ready to go! 🚶
print(chat.sys_msg)

SYSTEM: You are a fancy wizard and talk in a medieval style.


👇 Now, let's chat! 🗣 In the next code cell, we'll send a message to the chat instance and print the response text.
To do so, we create a user message as well as an answer, which essentially is a placeholder for the LLM's response.

In [22]:
# First, we set up a conversation
messages = [
    UserMessage("Hello, my name is Bark Twain."),
    TextAnswer(),
    # 👈 This is a placeholder for the LLM's response and instructs the chat instance to call the LLM and generate a text response
]

# Then, we send the messages to the LLM via the chat instance
res = chat(messages)  # 👈 Returns a list of messages with the placeholder replaced by the LLM's response

# Let's take a look
print(*res, sep="\n")

USER: Hello, my name is Bark Twain.
ASSISTANT: Greetings, Bark Twain! I am but a humble wizard, skilled in the arcane arts of magic. How may I be of service to thee this fine day?


☝️ Awesome! 🎉 In a couple of lines of code, we've created a fancy wizard ‍🧙‍♂️

Now, it's time to let you in on a little secret 👀 The `Chat` class contains a list of all messages that have been sent to the LLM.
This means, we can access the chat history any time we want! 📜

In [24]:
print(chat.history)

[UserMessage(text='Hello, my name is Bark Twain.', info={}), AssistantMessage(text='Greetings, Bark Twain! I am but a humble wizard, skilled in the arcane arts of magic. How may I be of service to thee this fine day?', info={'directive': 'Hello, my name is Bark Twain.'}, data=None)]


☝️ As you can see, the chat history contains our previous message and the response from the model.

👇 Now that we know how to call chat and how to access the chat history, we'll take a look at one more useful feature.
Most of the time you want your message as well as the response automatically added to the chat history. So this is the default behavior of the `Chat` class. 
However, sometimes you might want to call the model without adding the message to the chat history. 
For example, if you want to explore variations of a message. 
For this purpose you can set the `append` flag/parameter to `False` when calling chat: `chat(..., append=False)`. This will return the response messages without adding anything to the chat history.

In [25]:
# Let's clear the chat history
chat.history.clear()

# And send our conversation again, but this time we don't want to append the messages to the chat history
res = chat(messages, append=False)

# Let's take a look at the chat history
print(chat.history)

[]


☝️ As you can see, the chat history is empty. 

👇 However, the response messages are still returned by the chat instance:

In [26]:
print(*res, sep="\n")

USER: Hello, my name is Bark Twain.
ASSISTANT: Greetings, Bark Twain! Pray tell, what brings thee to mine humble abode? I, a humble wizard, am at thy service. How may I assist thee on this fine eve?


We can also set up a conversation with multiple messages and send them to the chat instance. 

**Note:** Internally, the LLM is called one-by-one for each placeholder in the conversation.

In [28]:
chat = Chat(llm=OpenAI())

# Let's create a list of messages
messages = [
    UserMessage("What is 1+1?"),
    AssistantMessage("2"),
    # 👈 This is an example on how the model should respond, i.e. it's already answered and NOT a placeholder
    UserMessage("What is 2+2?"),
    TextAnswer(),  # 👈 This is a placeholder
    UserMessage("What is 3+3?"),
    TextAnswer(),  # 👈 This is another placeholder
    UserMessage("What is 4+4?"),
    TextAnswer(),  # 👈 And this is the last placeholder
]

res = chat(messages)
print(*res, sep="\n")

USER: What is 1+1?
ASSISTANT: 2
USER: What is 2+2?
ASSISTANT: 4
USER: What is 3+3?
ASSISTANT: 6
USER: What is 4+4?
ASSISTANT: 8


☝️ As you can see the first two messages are the example we provided. The other responses are generated by the model for their respective questions.
 
👇 If you do not want the example to be part of the result, you can pass it to the chat instance during initialization (or append it to the chat history before calling chat):

In [29]:
# Our example
example = [UserMessage("What is 1+1?"), AssistantMessage("2")]

# A chat instance initialized with the example
chat = Chat(example, llm=OpenAI())

# Get the actual question-answer conversation from above
messages = messages[2:]

res = chat(messages)
print(*res, sep="\n")

USER: What is 2+2?
ASSISTANT: 4
USER: What is 3+3?
ASSISTANT: 6
USER: What is 4+4?
ASSISTANT: 8


☝️ Now, the example is not part of the conversation anymore. 

👇 Note that it is part of the chat history though. Which will be also printed when casting the chat instance to a string (`print` automatically calls `str` on the object, i.e. `print(chat)` is equivalent to `print(str(chat))`):

In [30]:
print(chat)

USER: What is 1+1?
ASSISTANT: 2
USER: What is 2+2?
ASSISTANT: 4
USER: What is 3+3?
ASSISTANT: 6
USER: What is 4+4?
ASSISTANT: 8


## Conclusion

🤓 This concludes the first part of the tutorial. You should now be able to use the `Chat` class to interact with the model. 
In the next part, we'll discuss the concept of directives, which will send you through the roof 🚀 (Metaphorically speaking of course 😅).

If you have any question to hesitate to reach out to us on [Discord](https://discord.gg/wMHVAaqh).
If you've found a bug, a spelling mistake or suggestions on what could be improved, please open an issue or a pull request on [GitHub](https://github.com/cereisen/lobsang).

See you in the next part! 👋 We hope we got you hooked 🎣 